In [1]:
# pipenv install pandas plotly scikit-learn optuna ipywidgets ipykernel nbformat gradio

In [2]:
import pandas as pd 
import numpy as np
import plotly.express as px 
pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(suppress=True, precision=10)

from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer

import optuna

### Carregar os dados

In [3]:
df_clientes = pd.read_csv(r'.\datasets\clientes.csv')

In [4]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 23.6+ KB


In [5]:
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
0,Comércio,713109.95,12,Rio de Janeiro,6,1
1,Comércio,790714.38,9,São Paulo,15,0
2,Comércio,1197239.33,17,São Paulo,4,9
3,Indústria,449185.78,15,São Paulo,6,0
4,Agronegócio,1006373.16,15,São Paulo,15,8
5,Serviços,1629562.41,16,Rio de Janeiro,11,4
6,Serviços,771179.95,13,Vitória,0,1
7,Serviços,707837.61,16,São Paulo,10,6
8,Comércio,888983.66,17,Belo Horizonte,10,1
9,Indústria,1098512.64,13,Rio de Janeiro,9,3


In [6]:
df_clientes.tail(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
490,Indústria,215580.61,11,Belo Horizonte,7,3
491,Serviços,1050776.57,14,Vitória,8,0
492,Comércio,785671.05,15,Vitória,9,2
493,Serviços,658330.45,20,Belo Horizonte,4,8
494,Agronegócio,1643153.26,14,Rio de Janeiro,10,1
495,Serviços,1581841.42,17,Rio de Janeiro,8,2
496,Indústria,1291309.57,9,São Paulo,6,9
497,Serviços,2211489.85,10,Belo Horizonte,10,0
498,Agronegócio,1460860.46,12,Rio de Janeiro,5,3
499,Indústria,173684.43,13,Belo Horizonte,4,9


In [7]:
df_clientes.describe()

,faturamento_mensal,numero_de_funcionarios,idade,inovacao
count,500.00,500.00,500.00,500.00
mean,1026715.63,13.69,9.25,4.39
std,420609.46,3.12,2.96,2.90
min,18421.22,2.00,0.00,0.00
25%,763253.58,12.00,7.00,2.00
50%,1022957.08,14.00,9.00,4.00
75%,1295888.52,16.00,11.00,7.00
max,2390677.22,21.00,16.00,9.00


### Preparar Dados para treinamento e execução do modelo GMM

In [8]:
# Selecionar as ccolunas relevantes para a clusterizacao

X = df_clientes.copy()

# Separar variaveis por tipo

numeric_features =  ['faturamento_mensal', 'numero_de_funcionarios', 'idade']
categorical_features = ['localizacao', 'atividade_economica']
ordinal_feature = ['inovacao']

# Transformers

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()
ordinal_transformer = OrdinalEncoder()

# preprocessor 

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features),
    ('ord', ordinal_transformer, ordinal_feature)
])

# Transformar os dados

X_transformed = preprocessor.fit_transform(X)

In [9]:
# Visualizar X_transformed 
X_transformed

array([[-0.7463449774, -0.5417919104, -1.1005884861, ...,  0.          ,
         0.          ,  1.          ],
       [-0.5616554761, -1.5035526981,  1.9434485069, ...,  0.          ,
         0.          ,  0.          ],
       [ 0.4058265391,  1.0611427358, -1.7770411512, ...,  0.          ,
         0.          ,  9.          ],
       ...,
       [ 2.8196246022, -1.1829657689,  0.2523168441, ...,  0.          ,
         1.          ,  0.          ],
       [ 1.0332141129, -0.5417919104, -1.4388148187, ...,  0.          ,
         0.          ,  3.          ],
       [-2.0301148644, -0.2212049812, -1.7770411512, ...,  1.          ,
         0.          ,  9.          ]], shape=(500, 12))

### Treinar o modelo GMM

In [10]:
# Criar funcao para executar no optuna
def gmm_objective(trial):
    # Definir hiperparametros a serem ajustados
    n_components = trial.suggest_int('n_components', 3,10)
    covariance_type = trial.suggest_categorical('covariance_type', ['full', 'tied', 'diag', 'spherical'])
    # Instanciar o modelo GMM com os hiperparâmetros
    gmm = GaussianMixture(n_components=n_components, covariance_type=covariance_type, random_state=51)
    # Treinar o modelo nos dados
    gmm.fit(X_transformed)
    # Calculando o BIC (Bayesian Information Criteria)
    bic_gmm = gmm.bic(X_transformed)
    
    return bic_gmm

In [11]:
# Criar estudo do optuna 
search_space = {'n_components': [3,4,5,6,7,8,9,10], 'covariance_type': ['full', 'tied', 'diag', 'spherical']}
sampler = optuna.samplers.GridSampler(search_space=search_space)
estudo_gmm = optuna.create_study(direction='maximize', sampler=sampler)

[I 2025-10-28 13:43:02,271] A new study created in memory with name: no-name-c117373d-818f-40c5-aff9-3d1b0484892a


In [12]:
# Executar o Optuna para otimizar os hiperparametros

estudo_gmm.optimize(gmm_objective, n_trials=32)

[I 2025-10-28 13:43:05,598] Trial 0 finished with value: -177.47638674251778 and parameters: {'n_components': 6, 'covariance_type': 'tied'}. Best is trial 0 with value: -177.47638674251778.
[I 2025-10-28 13:43:05,651] Trial 1 finished with value: -23479.731809069122 and parameters: {'n_components': 9, 'covariance_type': 'diag'}. Best is trial 0 with value: -177.47638674251778.
[I 2025-10-28 13:43:05,687] Trial 2 finished with value: -239.16103285256702 and parameters: {'n_components': 5, 'covariance_type': 'tied'}. Best is trial 0 with value: -177.47638674251778.
[I 2025-10-28 13:43:05,834] Trial 3 finished with value: -16669.227646727628 and parameters: {'n_components': 5, 'covariance_type': 'full'}. Best is trial 0 with value: -177.47638674251778.
[I 2025-10-28 13:43:05,858] Trial 4 finished with value: 1570.0096069322854 and parameters: {'n_components': 3, 'covariance_type': 'diag'}. Best is trial 4 with value: 1570.0096069322854.
[I 2025-10-28 13:43:05,948] Trial 5 finished with va

In [13]:
# Melhor configuração obtida pelo optuna
best_config = estudo_gmm.best_params

In [14]:
# Instanciar o modelo com melhores parametros
best_gmm = GaussianMixture(n_components=best_config['n_components'], covariance_type=best_config['covariance_type'], random_state=51)
best_gmm.fit(X_transformed)

# Calcular o BIC do melhor modelo
best_bic = best_gmm.bic(X_transformed)

In [15]:
# Mostrar melhores parâmetros

print(f"Quantidade ideal de componentes: {best_config['n_components']}")
print(f"Tipo ideal de Covariancia: {best_config['covariance_type']}")
print(f"BIC do melhor modelo: {best_bic}")

Quantidade ideal de componentes: 3
Tipo ideal de Covariancia: spherical
BIC do melhor modelo: 13517.954765258872


### Retornar Clusters do Melhor modelo GMM

In [16]:
# Executar predict para retornar os clusters 
clusters_gmm = best_gmm.predict(X_transformed)

In [17]:
# Visuzalizar resultado dos clusters
clusters_gmm

array([0, 0, 1, 0, 1, 2, 0, 1, 0, 2, 2, 2, 1, 2, 1, 1, 1, 0, 1, 2, 0, 0,
       2, 2, 0, 0, 1, 1, 0, 0, 0, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2,
       0, 1, 2, 0, 1, 1, 1, 0, 0, 2, 0, 0, 0, 0, 2, 1, 1, 2, 0, 1, 1, 2,
       2, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 2, 2, 1, 1, 0, 0, 1, 2, 1, 1, 0,
       0, 1, 0, 2, 2, 0, 0, 1, 1, 1, 1, 2, 0, 0, 2, 2, 0, 2, 1, 2, 2, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 2, 2, 0, 0, 2, 2, 1, 0, 1, 2, 2, 1, 1, 0,
       0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 2, 2, 1,
       0, 1, 1, 1, 0, 0, 2, 1, 1, 1, 2, 2, 0, 0, 1, 1, 1, 1, 0, 2, 1, 0,
       1, 1, 1, 1, 0, 2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 0, 2, 1, 0, 1, 0, 0,
       2, 2, 2, 1, 1, 0, 0, 1, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0, 0,
       1, 2, 1, 1, 1, 1, 0, 1, 0, 2, 0, 2, 1, 1, 0, 1, 2, 2, 1, 2, 1, 2,
       0, 0, 0, 1, 2, 1, 1, 1, 1, 0, 0, 0, 1, 2, 1, 1, 1, 0, 2, 1, 0, 1,
       0, 2, 1, 1, 0, 2, 1, 1, 0, 1, 1, 2, 2, 1, 2, 1, 2, 1, 0, 0, 1, 1,
       1, 1, 2, 0, 0, 1, 0, 2, 1, 0, 1, 2, 1, 0, 2,

In [18]:
# Gerar as probabilidades de cada registro esta em um cluster
cluster_gmm_prob  =  best_gmm.predict_proba(X_transformed)

In [19]:
# Mostrar as probabilidades
cluster_gmm_prob

array([[0.9999798986, 0.          , 0.0000201014],
       [0.9999999957, 0.          , 0.0000000043],
       [0.          , 1.          , 0.          ],
       ...,
       [0.9999999977, 0.          , 0.0000000023],
       [0.053478558 , 0.0000000173, 0.9465214247],
       [0.          , 1.          , 0.          ]], shape=(500, 3))

In [20]:
# Criar uma coluna 'Cluster' no DF
df_clientes['Cluster'] = clusters_gmm.astype(int)

In [21]:
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,Cluster
0,Comércio,713109.95,12,Rio de Janeiro,6,1,0
1,Comércio,790714.38,9,São Paulo,15,0,0
2,Comércio,1197239.33,17,São Paulo,4,9,1
3,Indústria,449185.78,15,São Paulo,6,0,0
4,Agronegócio,1006373.16,15,São Paulo,15,8,1
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,2
6,Serviços,771179.95,13,Vitória,0,1,0
7,Serviços,707837.61,16,São Paulo,10,6,1
8,Comércio,888983.66,17,Belo Horizonte,10,1,0
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,2


### Visualizar resultados

In [22]:
# Cruzar dados de idade, faturamento mensal e clusters
px.scatter(df_clientes, x='idade', y='faturamento_mensal', color='Cluster')

In [23]:
# Cruzar dados de faturamento_mensal, inovacao e clusters
px.scatter(df_clientes, x='inovacao', y='faturamento_mensal', color='Cluster')

In [24]:
# Cruzar dados de idade, inovacao e clusters
px.scatter(df_clientes, x='inovacao', y='idade', color='Cluster')

### Salvar modelo

In [25]:
import joblib 

joblib.dump(best_gmm, 'modelo_clusterizacao_gmm.pkl')
joblib.dump(preprocessor, 'preprocessor_clusterizacao_gmm.pkl')

['preprocessor_clusterizacao_gmm.pkl']

### Aplicação de geração dos clusters

In [26]:
import gradio as gr 

modelo = joblib.load('modelo_clusterizacao_gmm.pkl')
pipeline = joblib.load('preprocessor_clusterizacao_gmm.pkl')

In [27]:
def clustering(arq):
    df_empresas = pd.read_scv(arq.name)
    X_transformed = pipeline.fit_transform(df_empresas)
    modelo.fit(X_transformed)
    clusters_empresas = modelo.predict(X_transformed)
    df_empresas['Cluster'] = clusters_empresas.astype(int)
    df_empresas.to_csv('.\clusters_empresas.csv', index= False)
    return '.\clusters_empresas.csv'
    

In [28]:
app = gr.Interface(
    clustering,
    gr.File(file_types=[".csv"]),
    "file"
)

app.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
